# Landsat LAI Example - Interactive

In [1]:
import ee
import folium

import openet.lai.landsat

# ee.Authenticate()
ee.Initialize()


In [2]:
viridis_palette = ['#440154', '#433982', '#30678D', '#218F8B', '#36B677', '#8ED542', '#FDE725']
ndvi_palette = ['EFE7E1', '003300']

image_size = 768


In [3]:
landsat_id = 'LC08_042034_20180705'
# landsat_id = 'LC08_044033_20170716'  # pytest scene

landsat_img = ee.Image(f'LANDSAT/LC08/C02/T1_L2/{landsat_id}')
landsat_crs = landsat_img.select('SR_B3').projection().getInfo()['crs']

map_xy = [-120, 37]
# map_xy = landsat_sr_img.geometry().centroid().getInfo()['coordinates']
# print(map_xy)

max_zoom = 12


In [4]:
# "Prep" the Landsat image
# This same functionality is in the Landsat module/class
# Note, the reflectance values are intentionally being scaled by 10000 to
#   match the original Collection 1 SR scaling used in the training
input_bands = ee.Dictionary({
    'LANDSAT_5': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'],
    'LANDSAT_7': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'],
    'LANDSAT_8': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'],
    'LANDSAT_9': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'],
})
output_bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'qa']
spacecraft_id = ee.String(landsat_img.get('SPACECRAFT_ID'))
prep_img = (
    landsat_img
    .select(input_bands.get(spacecraft_id), output_bands)
    .multiply([0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 0.0000275, 1])
    .add([-0.2, -0.2, -0.2, -0.2, -0.2, -0.2, 0])
    .divide([0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 1])
    .set({
        'system:index': landsat_img.get('system:index'),
        'system:time_start': landsat_img.get('system:time_start'),
        'system:id': landsat_img.get('system:id'), 
        'SPACECRAFT_ID': spacecraft_id,
        'SOLAR_AZIMUTH_ANGLE': ee.Number(landsat_img.get('SUN_AZIMUTH')),
        'SOLAR_ZENITH_ANGLE': ee.Number(landsat_img.get('SUN_ELEVATION'))
            .multiply(-1).add(90),
    })
)

# Generate the LAI image
# Right now the input image needs to be unscaled (raw) but renamed
output_img = openet.lai.model.get_lai_image(
    prep_img, sensor=landsat_id.split('_')[0], nonveg=True,
)


In [5]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr="Map Data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
        show=show
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [6]:
my_map = folium.Map(location=[map_xy[1], map_xy[0]], zoom_start=max_zoom, height=image_size)

# Add the layers to the map
my_map.add_ee_layer(
    prep_img.select(['red', 'green', 'blue']), {'min': 0, 'max': 3000}, 'True Color', True)
my_map.add_ee_layer(
    prep_img.normalizedDifference(['nir', 'red']).select(['nd'], ['ndvi']), 
    {'min': -0.1, 'max': 0.9, 'palette': ndvi_palette}, 'NDVI', True)
my_map.add_ee_layer(
    output_img.select(['LAI']).reproject(crs=landsat_crs, scale=30), 
    {'min': 0, 'max': 4, 'palette': viridis_palette}, 'LAI', True)

my_map.add_child(folium.LayerControl())
display(my_map)